In [6]:
get_ipython().run_line_magic('pylab', 'inline')

Populating the interactive namespace from numpy and matplotlib


In [7]:
#from tinc import *
import tinc
import os
import numpy as np
import clex as cl


In [8]:
#Enlarge the jupyter notebook so that plots are easier to see 
from IPython.core.display import display, HTML
#display(HTML("<style>.container { width:80% !important; }</style>"))

In [9]:


eci_index_1 = tinc.Parameter("eci_index_1", "eci_index")
eci_index_1.values = np.array(list(range(253)))

eci_index_2 = tinc.Parameter("eci_index_2", "eci_index")
eci_index_2.values = np.array(list(range(253)))

graph_buffer = tinc.DiskBufferImage("graph", "out.png", "eci_covariance_images")

In [10]:
ps = tinc.ParameterSpace("ps")
ps.register_parameters([eci_index_1, eci_index_2])
ps.enable_cache()

Validating json with /home/derick/anaconda3/envs/py37/lib/python3.7/site-packages/tinc/tinc_cache_schema.json


In [11]:
results = cl.run_eci_monte_carlo('corr_and_formation_energy.json', 0.004, 10000, "./")
sampled_eci = np.array(results["sampled_eci"])

/home/derick/anaconda3/envs/py37/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:528: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.00012856272581029948, tolerance: 8.418864644900934e-05
  tol, rng, random, positive)
/home/derick/anaconda3/envs/py37/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:528: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.00013019349025522597, tolerance: 8.416760210014582e-05
  tol, rng, random, positive)
/home/derick/anaconda3/envs/py37/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:528: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.0005670752607485618, tolerance: 8.418864644900934e-05
  tol, rng, random, positive)
/home/derick/anaconda3/envs/py37/lib/python3.7/site-packages/sklearn

Acceptance Probability is: 0.176666


In [12]:
def make_graph(eci_index_1, eci_index_2):
    eci_selection_1 = sampled_eci[:,int(eci_index_1)]
    eci_selection_2 = sampled_eci[:,int(eci_index_2)]
    
    fig = cl.plot_eci_covariance(eci_selection_1, eci_selection_2)
    fig.set_size_inches(18, 10)
    image_path = os.path.join("./eci_covariance_images", 'eci_index_%d_%d.png'%(eci_index_1, eci_index_2))
    fig.savefig(image_path)
    file = open(image_path, 'rb')
    return file.read()


def value_changed(value):
    imagedata = ps.run_process(make_graph)
    graph_buffer.data = imagedata

eci_index_1.register_callback(value_changed)
eci_index_2.register_callback(value_changed)


In [13]:
#print(A.values,B.values,kT.values)

#currently not using this display approach
from ipywidgets import GridspecLayout
grid = GridspecLayout(3, 2, height='300px')
grid[:, 1] = graph_buffer.interactive_widget()
grid[0, 0] = A.interactive_widget()
grid[1, 0] = B.interactive_widget()
grid[2, 0] = kT.interactive_widget()
grid


In [14]:
from ipywidgets import Button, HBox, VBox

box = VBox([eci_index_1.interactive_widget(), eci_index_1.interactive_control(), eci_index_2.interactive_widget(), eci_index_2.interactive_control(), graph_buffer.interactive_widget()])
graph_buffer._interactive_widget.width = "80%"
box